# POZAlabs Task
- MusicVAE 구현
- Groove MIDI Dataset을 이용하여 4마디에 해당하는 드럼 샘플 추출 과정 수행
- 깃헙 코드: https://github.com/magenta/magenta/tree/master/magenta/models/music_vae
- 관련 논문: https://arxiv.org/pdf/1803.05428.pdf
- 관련 데이터: https://magenta.tensorflow.org/datasets/groove

## Commentary
- 미디라는 데이터는 음악이 어떻게 연주되어야 하는지 나타내는 악보와 비슷한 개념으로 이해하시면 됩니다. 어느 시점에 어떤 악기를 연주하여야 하는지가 나와있는 데이터이며 .midi의 확장자명을 가지고 있습니다.
- 설명드린 코드에서는 해당 미디를 학습에 이용하기 위하여 벡터로 변환하는 전처리 과정이 필요합니다. 이러한 전처리 과정을 거치면 tfrecord형식으로 저장되게 되며 이것을 학습하는 형태입니다.
- 추가적으로, 해당 과정(전처리 -> 학습 -> 생성)의 프로세스에 대하여 자유로운 형식으로 정리하여 공유주시면 감사하겠습니다. (학술적인 내용이 포함되면 좋습니다!)
- 과제에 필요한 라이브러리의 dependency 이슈가 있습니다. 필요한 라이브러리 설치가 안되는 경우 https://github.com/magenta/magenta/issues/2005 를 참고하시어 진행하시길 바랍니다.
- 과제는 어떠한 방식으로 모델을 다루는지를 보기 위함이며, 과제를 100% 완수하지 못하셔도 됩니다.
- 과제와 관련한 모든 문의는 해당 메일로 회신 부탁드립니다.

## Process
1. Set Environment (1 hour)
2. Explore Background (5 hours)
3. Review Reference Code
4. Analyze Data
5. Define Target Process

<hr>

## Set Environment
- Start with MacOS Monterey 12.4
- Make conda env
- Try to install Magenta library (08-19 15:00-16:00, 1 hour)
- Continue with Colab instead of MacOS

<hr style="border-style:dotted">

### Magenta Issues

> Could not find a version that satisfies the requirement tensorflow==2.9.1
- `tensorflow-macos==2.9.1` 설치 후에도 동일 에러 발생
- 에러 처리를 위해 과거 Issu 내역과 검색 결과 참고
- https://github.com/magenta/magenta/issues/1867
- https://github.com/magenta/magenta/issues/1886
- https://stackoverflow.com/questions/62948255/magenta-installation-using-pip-install-magenta-having-errors-even-on-fresh-env
- 시간 내 과제 해결을 위해선 해당 에러 처리를 고집하는 것보다 되는 환경에서 우선 수행하는 것이 바람직하다 판단하여 Colab 환경에서 진행

In [ ]:
!pip install magenta==2.1.4

<hr>

## Explore Background
- Find prior knowledge for VAE (08-19 16:00-18:00, 2 hours)
- Review suggested [reference](https://arxiv.org/pdf/1803.05428.pdf) (08-19 18:00-21:00 3 hours)

<hr style="border-style:dotted">

### VAE(Auto-Encoding Variational Bayes)

<img src="https://user-images.githubusercontent.com/24144491/50323466-18d03700-051d-11e9-82ed-afb1b6e2666a.png" width="50%">

- VAE는, Encoder로 차원축소를 하기 위한 목적의 Audoencoder와 반대로, Decoder로 새로운 데이터를 생성하기 위한 목적
- $x_i$에 대해 Gaussian Encoder를 통과한 출력 결과인 $\mu, \sigma$를 가지고 정규 분포를 생성, 해당 분포에서 $z_i$를 샘플링
- $z_i$를 가지고 Bernoulli Decoder를 통과하면 $x_i$와 동일한 출력 결과 $p_i$을 생성
- 샘플링에 대한 역전파 과정을 수행하기 위해 Reparameterization Trick을 사용 ($N(0,1)$을 따르는 정규 분포 $\epsilon_i$에 $\sigma_i^2$를 곱하고 $\mu_i$를 더함)
- Decoder가 생성한 데이터가 학습 데이터와 닮은 특징을 가짐

<br>

#### Backpropagation을 위한 Sampling 과정

```python
import keras.backend as K

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0],latent_dim),mean=0., stddev=1.)
    return z_mean + K.exp(z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])
```

<br>

#### Loss Function

<img src="https://user-images.githubusercontent.com/24144491/50323472-1a016400-051d-11e9-86b7-d8bf6a1a880f.png" width="50%">

- Loss Function은 Reconstruction Error(복원 오차)와 Regularization 과정에서 도출되는 KL divergence(제약 조건)의 합으로 구해짐
- Reconstruction Error는 확률을 정규 분포로 가정하면 MSE, 이항 분포로 가정하면 CrossEntropy로 가짐 (일반적으론 베르누이로 지정)
- Regularization은 인코더를 통과해 나온 확률 분포가 정규 분포로 가정된 $p(z)$를 따르도록 하기 위해 두 분포 간 거리(KL divergence)를 최소화하는 과정
- 증명 과정을 통해 도출된 likelihood에 대한 ELBO 식 $\log{p_{\theta}(x^{(i)})\ge{L(x^{(i)},\theta,\phi)}}$에서 lower bound인 Loss Function을 maximize하는,   
  디코더의 파라미터 $\theta$와 인코더의 파라미터 $\phi$를 찾기 위함이 목적
- 최적화 과정에서는 Loss Function을 최소화하기 위해 Reconstruction Error에 마이너스를 곱해서 계산

<br>

#### Loss Function 도출 과정

```python
import keras.backend as K

def vae_loss(self, x, z_decoded):
    x = K.flatten(x)
    z_decoded = K.flatten(z_decoded)
    xent_loss = keras.metrics.binary_crossentropy(x,z_decoded)
    kl_loss   = -5e-4*K.mean(1+z_log_var-K.square(z_mean)-K.exp(z_log_var),axis=-1)
    return K.mean(xent_loss + kl_loss)
```

<br>

#### Reference
- [KAIST 스마트설계연구실 강남우 교수(전 숙명여대)의 딥러닝과 설계 강의](https://youtu.be/GbCAwVVKaHY)
- [VAE(Auto-Encoding Variational Bayes) 직관적 이해](https://taeu.github.io/paper/deeplearning-paper-vae/)

<hr style="border-style:dotted">

### A Hierarchical Latent Vector Model for Learning Long-Term Structure in Music

#### 1. Introduction
- 생성 모델링은 데이터 x를 생성하기 위한 확률 분포 $p(x)$를 도출하는 과정
- 최근(2018)의 생성 모델링 연구는 복잡한 구조를 사용하는 DNN을 통해 가속화되었으며,   
  해당 연구에서는 VAE 등 deep latent variable 모델에 주목
- 여전히 긴 시퀀스에 대해 deep latent variable 모델은 성과를 내지 못하고 있는데,   
  이를 보완하기 위해 계층적 순환 디코더를 포함하는 오토인코더를 제안
- 제안할 모델은 전체 latent vector를 인코딩하여, baseline인 flat decoder RNN보다 긴 시퀀스에 효과적

#### 2. Background

##### 2-2. Reccurent VAEs

<img src="https://i.imgur.com/PQKoraX.png" width="50%">

- 인코더는 입력 시퀀스에 대한 hidden states를 생성하는 RNN 모델
- latent code $z$의 분포는 $h_T$로 설정되며, 동시에 디코더 RNN의 초기 상태로 사용됨
- 출력 시퀀스 $y$를 발생시키는 디코더는 일반적인 VAE처럼 $p(z)$에 근사해지는 $q_{\lambda}(z|x)$를 학습함

#### 3. Model

##### 3.1. Bidirectional Encoder
- 인코더에 대해 2계층 양방향 LSTM 구조를 사용하여 마지막 상태 벡터 $\overrightarrow{h_T}, \overleftarrow{h_T}$를 도출
- 두 벡터를 결합한 $h_T$를 2개의 fully-connected 레이어에 넣어서 $\mu$와 $\sigma$를 생성 ($\ \mu=W_{h\mu}h_T+b_\mu,\ \sigma=\log{(\exp{(W_{h\sigma}h_T+b_{\sigma})+1})}\ $)
- 모든 레이어에 대해 2048개의 state size와 512 latent dimension을 가지는 LSTM이 입력 시퀀스에 대한 문맥으로부터 $\mu$와 $\sigma$를 도출하게 됨

##### 3.2. Hierachical Decoder
- 2계층 디코더 RNN에 대해 각각의 계층에서 1024 units으로 구성된 LSTM을 사용
- 디코더 RNN은 초기 상태로 latent vector $z$가 설정되고, autoregressive하게 출력 시퀀스를 생성
- 하지만, 이전 연구에서 단순히 쌓인 구조의 RNN은 긴 시퀀스에 대해 낮은 재현율을 보이며,   
  이것이 출력 시퀀스를 생성하는데 있어서 latent state의 영향력이 사라지는 것에 있다고 판단
- 해당 문제를 완화시키기 위해 계층적 RNN 디코더를 사용하는데,   
  입력 시퀀스 $\text{x}$를 $U$개의 서브시퀀스로 나눌 경우에, tanh 활성화 함수가 적용된 fc 레이어를 통과시킨 $z$는 각각의 서브시퀀스와 대응되며,   
  해당 임베딩 벡터 집합 $\text{c}$의 원소가 각각의 fc 레이어를 지나면 $y_u$의 내부 요소에 해당하는 softmax 결과를 생성
- 위 그림을 참고하여 음악을 예로 들 경우, latent vector가 conductor를 지나면서 마디를 생성하고,   
  각각의 마디에서 RNN 모델을 통해 autoregressive하게 멜로디를 생성하여 전체적으로 latent vector의 영향력이 높아짐

##### 3.3. Multi-Stream Modeling
- 음악은 텍스트와 다르게 다양한 악기에 대한 시퀀스가 동시에 주어질 수 있기 때문에,   
  모델링에서 각각의 stream 간 복잡한 의존성을 고려할 필요가 있음
- 3개의 출력 토큰(drum, bass, melody)을 가지는 MusicVAE를 적용하고,   
  계층적 디코더 모델에서는 세 가지 요소를 직교 차원으로 규정하여 각각의 악기에 따라 분리된 디코더 RNN을 사용
- baseline인 flat 디코더에서는 단일 RNN을 사용하고, 악기 별 softmax를 도출하기 위해 출력 결과를 분리

#### 5. Experiments

##### 5.1. Data and Training
- 데이터로, 각각의 악기에 대한 노트와 미터 정보가 담긴 MIDI 파일을 사용
- 학습 데이터로 약 1억 5천만개의 파일을 수집하고, 16마디의 멜로디, 베이스, 드럼 패턴 등을 추출
- 멜로디의 경우 128개의 피치에 대한 note-on 토큰과 note-off, rest 토큰으로 구성된 130 차원의 벡터 공간으로 구성
- 드럼 패턴의 경우 61개의 드럼 분류를 9개의 표준 클래스에 맵핑하고 512개의 범주형 토큰으로 가능한 모든 조합을 표시
- 모든 모델은 $10^{-3}$에서 $10^{-5}$ 사이의 lr를 적용한 Adam 옵티마이저와 512개의 배치 단위로 학습

##### 5.2. Short Sequences

<img src="https://d3i71xaburhd42.cloudfront.net/2b050df9e24eb65b0d37f13c6eea1d29b4e316ce/6-Table1-1.png" width="30%" style="margin-left:50px">

- 순환 VAE를 통한 음악 시퀀스 생성이 가능함을 증명하기 위해 2마디의 짧은 시퀀스를 활용해 flat 디코더 모델을 학습시켰을 때,   
  입력값에 대해 높은 재현율을 보였지만, 16마디의 긴 시퀀스를 학습시켰을 경우 교사 강요와 샘플링 간 정확도 불일치가 27% 이상 증가

##### 5.3. Reconstruction Quality
- 계층적 디코더 모델의 경우 NSP 정확도와 노출 편향 가능성을 모두 고려해 학습하였고, 교사 강요 결과와의 불일치를 5-11% 사이로 좁힘
- multi-stream 데이터에 대해서 single-stream으로 활용한 flat 모델과 비교했을 때 압도적인 정확도 향상을 보였고, 교사 강요와도 매우 적은 차이를 보임

#### 6. Conclusion
- MusicVAE 모델을 제안하며, flat baseline보다 긴 시퀀스에서 뿐 아니라 사람을 통한 테스트에서도 높은 성능을 보임을 증명

---

## Data
- [Groove MIDI Dataset](https://magenta.tensorflow.org/datasets/groove)
- MIDI 형식으로 사람이 연주하는 드럼 소리를 녹음한 오디오 데이터셋
- 22,000개의 마디로 구성된 1,150개의 MIDI 파일을 포함
- 

In [ ]:
% 